In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

ModuleNotFoundError: No module named 'api_keys'

In [11]:
weather_df=pd.read_csv('../WeatherPy/output_data/cities.csv')

In [12]:
weather_df

,City,Cloudiness,Country,Date,Humidity (%),Latitude,Longitude,Max Temperature,Wind Speed (MPH)
0,Castro,0,BR,1583895273,92,-24.79,-50.01,61.16,5.50
1,Nanortalik,83,GL,1583895295,80,60.14,-45.24,20.61,8.08
2,Vaini,75,TO,1583895310,94,-21.20,-175.20,78.80,3.36
3,Hermanus,100,ZA,1583895310,83,-34.42,19.23,67.19,25.57
4,San Javier,0,ES,1583895311,76,37.81,-0.84,57.99,5.82
...,...,...,...,...,...,...,...,...,...
582,Dārāb,0,IR,1583895350,20,28.75,54.54,44.58,0.98
583,Sørland,100,NO,1583895350,70,67.67,12.69,39.16,10.20
584,Dossor,100,KZ,1583895350,53,47.53,52.98,41.29,11.90
585,Wad Rāwah,0,SD,1583895350,48,15.16,33.14,73.00,10.92


In [181]:
gmaps.configure(api_key=g_key)

In [182]:
# Store 'Latitude' and 'Longitude' into  locations 
locations = weather_df[["Latitude", "Longitude"]].astype(float)

# Store humidity
humidity = weather_df["Humidity (%)"].astype(float)

In [193]:
#Use Humidity has weight for heatmap
fig = gmaps.figure(center=(10,10), zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations ,weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [33]:
# Create new DataFrame fitting weather criteria

# - A max temperature lower than 80 degrees but higher than 70.
# - Wind speed less than 10 mph.
# - Zero cloudiness.

In [194]:
new_weather = pd.DataFrame(weather_df, columns = ["City", 'Latitude','Longitude',"Max Temperature", "Wind Speed (MPH)", "Cloudiness",'Humidity (%)'])

max_temp = (new_weather["Max Temperature"] <= 80) & (new_weather["Max Temperature"] > 70)
wind_speed = new_weather["Wind Speed (MPH)"] < 10
cloudiness = new_weather["Cloudiness"] == 0

new_weather=new_weather[max_temp & wind_speed & cloudiness]
new_weather.head()

,City,Latitude,Longitude,Max Temperature,Wind Speed (MPH),Cloudiness,Humidity (%)
8,Morondava,-20.28,44.28,73.67,5.79,0,75
89,San Ignacio,17.16,-89.07,78.80,6.93,0,74
127,Sharjah city,25.36,55.40,72.00,3.36,0,66
148,Distrikt Brokopondo,4.75,-55.17,73.44,7.14,0,81
190,Inhambane,-23.86,35.38,75.63,3.44,0,78


In [89]:
# Store into variable named hotel_df.
# Add a "Hotel Name" column to the DataFrame.
# Set parameters to search for hotels with 5000 meters.
# Hit the Google Places API for each city's coordinates.
# Store the first Hotel result into the DataFrame.
# Plot markers on top of the heatmap.

In [90]:
new_weather['Hotel Name']=''

In [174]:
# create a params dict holding information for radius and keyword (no 'Hotel' item found in documentation)
target_search='Hotel'
radius=5000
params = {
    "keyword": target_search,
    "radius": radius,
    "key": g_key
}
hotel_name_box=[]
# Loop through the df and search for hotels in radius of each lat/lng
for index, row in new_weather.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    lat = row['Latitude']
    lng = row['Longitude']
    city = row['City']

    # update address key value
    params['location'] = f"{lat},{lng}"

    # make request
    hotels_5000 = requests.get(base_url, params=params)

    #  print(hotels_5000.url)
    
    # convert to json
    hotels_5000 = hotels_5000.json()
    # store name (look through json layout first to determine index)
    #Use Try/Except to account for cities without hotel within radius 5000
    try:
        hotel_name=hotels_5000['results'][0]['name']
#         print(hotel_name)
        #gather data for marker label
#         hotel_name_marker= {'name': hotels_5000['results'][0]['name'], 'location': (f'({lat},{lng})')}
#         hotel_name_box.append(hotel_name_marker)
        
        new_weather.loc[index, 'Hotel Name'] = hotel_name
        
    except IndexError:
        print(f'There is no hotel near {city}.')

# Visualize to confirm lat lng appear
# new_weather.head()
                                                          

There is no hotel near Beloha.
There is no hotel near Qandala.


In [165]:
new_hotel=new_weather.dropna()

In [196]:
#Reset parameters for humidity and locations
#NA values have been removed so there are less spots now
humidity = new_hotel["Humidity (%)"].astype(float)
coordinates = new_hotel[["Latitude", "Longitude"]].astype(float)

In [176]:
# hotel_locations = [hotel['location'] for hotel in hotel_name_box]
# info_box_template = """
# <dl>
# <dt>Name</dt><dd>{name}</dd>
# </dl>
# """
# hotel_info = [info_box_template.format(**hotel) for hotel in hotel_name_box]

# marker_layer = gmaps.marker_layer(hotel_locations, info_box_content=hotel_info)
# fig = gmaps.figure()
# fig.add_layer(marker_layer)
# fig

In [177]:
hotel_name_box
#how do i remove the '' around the lat and lon

[{'name': 'Kimony Resort Hotel', 'location': '(-20.28,44.28)'},
 {'name': "Ka'ana Resort And Spa", 'location': '(17.16,-89.07)'},
 {'name': 'The Chedi Al Bait, Sharjah - a GHM hotel',
  'location': '(25.36,55.4)'},
 {'name': 'Matu Island', 'location': '(4.75,-55.17)'},
 {'name': 'Hotel Casa Do Capitao', 'location': '(-23.86,35.38)'},
 {'name': 'Hotel Maresta Lodge', 'location': '(-9.09,-78.58)'},
 {'name': 'The First Hotel', 'location': '(16.78,94.73)'},
 {'name': 'Hotel Polo Orchid', 'location': '(25.52,90.22)'},
 {'name': 'Hotel dos Camalotes', 'location': '(-21.7,-57.88)'},
 {'name': 'Hotel Xaguate', 'location': '(14.9,-24.5)'},
 {'name': 'Hotel Carabane', 'location': '(12.48,-16.55)'},
 {'name': 'Hotel Villa Kitzia Huacho', 'location': '(-11.1,-77.61)'},
 {'name': 'Hotel la siesta', 'location': '(-10.07,-78.15)'},
 {'name': 'Hotel VIP Praia', 'location': '(14.92,-23.51)'},
 {'name': 'ibis Presidente Prudente', 'location': '(-22.13,-51.39)'},
 {'name': 'Petit Rio Hotel', 'location':

In [197]:
fig = gmaps.figure(center=(10, 30),zoom_level=2)

markers = gmaps.marker_layer(coordinates)

# Add the layer to the map
fig.add_layer(markers)

heat_layer = gmaps.heatmap_layer(coordinates ,weights= humidity, 
                                 dissipating=False, max_intensity=100, point_radius = 3)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))